In [ ]:
import numpy as np
import pandas as pd
import processAc_tomo_funcs as pytomo
import pylook_extra as lc2
import h5py as h5

import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
mechdata = pytomo.read_hdf("../p5741_r.hdf5")
# mechdata.keys()

In [ ]:
# define settings
runname = "p5741"
acousticrun = 'run2'; # select the acoustic run to analyze after adjusting indexes (from figure 1) and the path
               
AcSettingsfile = "../acData/"+runname+"_"+acousticrun+".mat" # acoustic settings file (located in the current directory)
AcSyncFile = "./"+runname+"_sync_"+acousticrun+".hdf5" # sync file 
WF_path = "../acData/"+acousticrun+"/WF_" # where the WFs are                                                 

# Portion of the WF to analyze
idx2analyze = np.zeros((3,3,2),dtype='int') # 1st dim => transmitters, 2nd dim => receivers, 3rd dim => beg and end

"""RUN 1"""
# idx2analyze[0,:,:] = [[194, 372], [194, 305], [194, 305]]#T1
# idx2analyze[1,:,:] = [[165, 290], [190, 305], [230, 375]]#T2
# idx2analyze[2,:,:] = [[175, 330], [160, 310], [180, 340]]#T3

"""RUN 2"""
idx2analyze[0,:,:] = [[220, 455], [200, 300], [210, 300]]#T1
idx2analyze[1,:,:] = [[176, 276], [193, 293], [220, 340]]#T2
idx2analyze[2,:,:] = [[176, 300], [169, 278], [168, 310]]#T3

# Display waveforms sent by transmitter WhichTrans
WhichTrans = 2;           
# Time Range
TimeRange = np.array([11200, 12089]); # in seconds. Analyze acoustic data over that time range only. If empty, the whole run is analyzed
# AtWhichTimes = np.array([9774.5, 10182, 10605.74])
AtWhichTimes = np.array([11195.5, 11708, 12090])
NtoStack = 10
ref_type = 'absref'; #'absref', 'relref' or 'mixref';

In [ ]:
Filter = 0; #1 to filter, 0 not to filter
Filter_freq = np.array([0.25, 2]) #pass band filter 0.25MHz 2MHz
Filter_order = 256;
Filter_view = 1;

displayoptions = 1; # choose 0 to display all waveforms or 1 to display one set of waveforms over 100

ref_NrefWF = 50

# offset waveforms by Offset when multiple channels are used
Offset1 = 10000;
Offset2 = 10000;

# used for 'relref' or 'mixref'
threshold = 0.95;

FreqQ = 400e3; # frequency at which the amplitude is monitored
NZPad = 2048; #2^18; 2048 number of samples for zero padding before fft 

ref = [ref_type, ref_NrefWF]

In [ ]:
# plot mechanical data of interest within the considered run

# Find sample number corresponding to the beginning and end of the acoustic run
syncdata = pytomo.read_hdf(AcSyncFile)
FirstIdxAc = np.where(mechdata["time"].values > syncdata["acTime"].values[0])[0][0]
LastIdxAc = np.where(mechdata["time"].values > syncdata["acTime"].values[-1])[0][0]
idxAc = np.arange(FirstIdxAc,LastIdxAc)

In [ ]:
# %matplotlib widget
# fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
# axs[0].plot(mechdata["time"][idxAc], mechdata["effNS"][idxAc], color='red')
# axs[1].plot(mechdata["time"][idxAc], mechdata["Sync"][idxAc], color='blue')
# plt.tight_layout()
# plt.show()

In [ ]:
# show WFs at different times
# pytomo.ShowMeWFs(WF_path,AcSettingsfile,AcSyncFile,AtWhichTimes,NtoStack,Offset1,WhichTrans,idx2analyze)

In [ ]:
MaxInter, TimeShift, RmsAmp, Amp, RmsAmpRef, AmpRef, fullWFref, LocalAcTime, freqQAmp, maxAmp, maxFreq = pytomo.processAc(WF_path,AcSettingsfile,AcSyncFile,idx2analyze,ref, NtoStack,threshold,Offset2,displayoptions,Filter,TimeRange,NZPad,FreqQ)

In [ ]:
# TR = np.tile(["2", "5", "8"],(3,1))
# colors = np.array([["dodgerblue","blue","mediumblue"],["tomato","crimson","firebrick"],["lightgray","gray","black"]])

# %matplotlib widget
# fig, axs = plt.subplots(1, 1, figsize=(12, 5), sharex=True)
# for aa in range(3):
#     for bb in range(3):
#         axs.plot(LocalAcTime[:,aa], RmsAmp[:,bb,aa], label="T "+str(TR[bb,aa])+" --> "+"R "+str(TR.T[bb,aa]), color=colors.T[bb,aa])
# axs.set_xlabel("Time (s)")
# axs.set_ylabel("RMS Amp. (.)")
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
# %matplotlib widget
# fig, axs = plt.subplots(1, 1, figsize=(12, 5), sharex=True)
# for aa in range(3):
#     for bb in range(3):
#         axs.plot(LocalAcTime[:,aa], Amp[:,aa,bb], label=str(aa)+str(bb))
# axs.set_xlabel("Time (s)")
# axs.set_ylabel("Amp. (.)")
# plt.legend()
# plt.tight_layout()
# plt.show()

In [ ]:
RmsAmpI = pytomo.interpDat(mechdata["time"][idxAc], LocalAcTime, RmsAmp, idxAc, 3, 3)
TimeShiftI = pytomo.interpDat(mechdata["time"][idxAc], LocalAcTime, TimeShift, idxAc, 3, 3)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
for aa in range(3):
    for bb in range(3):
        axs[0].plot(mechdata["time"][idxAc], TimeShiftI[:,aa,bb], label=str(aa)+str(bb))
axs[0].legend()
axs[0].set_ylabel("Time Shift (.)")
# axs.set_ylim(0.6, 1.7)
axs[1].plot(mechdata["time"][idxAc], mechdata["effNS"][idxAc], "crimson")
axs[1].set_ylabel('$\sigma_{NS}$ (MPa)')
axs[1].set_xlabel('time (s)')
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
axs[0].plot(mechdata["time"][idxAc], TimeShiftI[:,1,1], label="T 2 --> R 2")
# axs[0].plot(mechdata["time"][idxAc], TimeShiftI[:,1,1], label="T 5 --> R 5")
# axs[0].plot(mechdata["time"][idxAc], TimeShiftI[:,2,2], label="T 8 --> R 8")
axs[0].legend()
axs[0].set_ylabel("Time Shift ($\mu s$)")
axs[1].plot(mechdata["time"][idxAc], mechdata["effNS"][idxAc], "crimson")
axs[1].set_ylabel('$\sigma_{eff\ NS}$ (MPa)')
axs[2].plot(mechdata["time"][idxAc], mechdata["perm2"][idxAc]*1e18, "k")
axs[2].set_ylabel('Perm. ($m^2$)')
axs[2].set_xlabel('time (s)')
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
axs[0].plot(mechdata["time"][idxAc], RmsAmpI[:,0,0], label="T 2 --> R 2")
axs[0].legend()
axs[0].set_ylabel("RMS Amp. (.)")
axs[1].plot(mechdata["time"][idxAc], mechdata["effNS"][idxAc], "crimson")
axs[1].set_ylabel('$\sigma_{NS}$ (MPa)')
axs[2].plot(mechdata["time"][idxAc], mechdata["perm2"][idxAc]*1e18, "k")
axs[2].set_ylabel('Perm. ($m^2$)')
axs[2].set_xlabel('time (s)')
plt.tight_layout()
plt.show()

In [ ]:
# fig, axs = plt.subplots(1, 1, figsize=(12, 8), sharex=True)
# axs.scatter(test[idxAc], RmsAmpIP[:,0,0], color="grey")
# axs.scatter(test[idxAc], RmsAmpIS[:,0,0], color="darkgrey")
# axs.set_ylabel('rms amplitude (.)')
# axs.set_ylim(170, 350)
# # ax2 = axs.twinx()
# # ax2.plot(mechdata["time_s"][idxAc], test[idxAc], "crimson")
# # ax2.set_ylabel('layer thickness detrend ($\mu m$)', color="crimson")
# axs.set_xlabel('layer thickness detrend ($\mu m$)')
# plt.tight_layout()
# plt.show()

In [ ]:
def saveData(runname, acousticrun, LocalAcTime, NtoStack, ref, datakeys, acdata):
    if len(TimeRange) == 0:
        filenamedata = "Results_"+runname+"_"+acousticrun+str(LocalAcTime[0,0])+"s-"+str(LocalAcTime[-1,-1])+"s_Stack"+str(NtoStack)+"WFs_"+ref[0]+"w_Amp.hdf5"
    else:
        filenamedata = "Results_"+runname+"_"+acousticrun+"_fullrun_Stack"+str(NtoStack)+"WFs_"+ref[0]+"w_Amp.hdf5"

    with h5.File(filenamedata, "w") as f:
        for aa, chanName in enumerate(datakeys):
            f.create_dataset(chanName, data=acdata[aa])

In [ ]:
datakeys = ["MaxInter", "TimeShift", "RmsAmp", "Amp", "RmsAmpRef", 
    "AmpRef", "fullWFref", "LocalAcTime", "freqQAmp", "maxAmp", 
    "maxFreq"]

saveData(runname, acousticrun, LocalAcTime, NtoStack, ref, datakeys, 
    acdata=[MaxInter, TimeShift, RmsAmp, Amp, RmsAmpRef, AmpRef, 
    fullWFref, LocalAcTime, freqQAmp, maxAmp, maxFreq])